# PYTS/BOSSVS classification algorithm test and measured accuracies
# Tested on transformed data

In [7]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import math
import sys 
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.metrics import f1_score

In [8]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/OFF_data/"

In [9]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)       

In [10]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [11]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):


    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
                a=test_index[i]
                Q_ON.append(a)
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]



if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/7):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)


(1089, 3, 37)
Accuracy for ON class:  100.0 %
Accuracy for OFF class:  67.71792677308095 %
False Positive rate:  32.28207322691904 %
False Negative rate:  0.0 %
F1 score:  0.7894862371209471
[['4FGL J0932+5306.csv', 36], ['4FGL J0940-1335.csv', 37], ['4FGL J1018+3540.csv', 55], ['4FGL J1208+5441.csv', 37], ['4FGL J1253+6242.csv', 42], ['4FGL J1351+1115.csv', 30], ['4FGL J1418-0233.csv', 39], ['4FGL J1419+5423.csv', 50], ['4FGL J1445-0326.csv', 39], ['4FGL J1445-1626.csv', 32], ['4FGL J1448+3608.csv', 35], ['4FGL J1450+5201.csv', 39], ['4FGL J1451+6355.csv', 34], ['4FGL J1514-0949.csv', 31], ['4FGL J1517+6525.csv', 36], ['4FGL J1518+4044.csv', 42], ['4FGL J1520+5546.csv', 39], ['4FGL J1520-0348.csv', 50], ['4FGL J1522+3144.csv', 39], ['4FGL J1526-0831.csv', 33], ['4FGL J1530+5736.csv', 43], ['4FGL J1532-1319.csv', 38], ['4FGL J1533+1855.csv', 43], ['4FGL J1533+3416.csv', 46], ['4FGL J1534+3716.csv', 47], ['4FGL J1535+5320.csv', 41], ['4FGL J1539+2743.csv', 40], ['4FGL J1539-1127.csv', 3

In [12]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f) and filename in errOFF :
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
        
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)       

In [13]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON

In [14]:

from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):

    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]
print(len(test_index))
print(len(np.unique(Q_ON)))

if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/8):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

(365, 3, 37)
Accuracy for ON class:  100.0 %
Accuracy for OFF class:  71.70899707230845 %
False Positive rate:  28.291002927691558 %
False Negative rate:  0.0 %
F1 score:  0.7995841887509093
73
100
[['4FGL J1707+1649.csv', 30], ['4FGL J2034+1154.csv', 34], ['4FGL J2150-1410.csv', 45], ['4FGL J2151+4156.csv', 28], ['4FGL J2153-1137.csv', 47], ['4FGL J2156+1818.csv', 42], ['4FGL J2156-0036.csv', 44], ['4FGL J2157+3127.csv', 39], ['4FGL J2158-1501.csv', 47], ['4FGL J2200+2138.csv', 29], ['4FGL J2202+4216.csv', 43], ['4FGL J2203+1725.csv', 32], ['4FGL J2204+0438.csv', 38], ['4FGL J2206-0032.csv', 39], ['4FGL J2207+4316.csv', 44], ['4FGL J2209-0451.csv', 38], ['4FGL J2211-1325.csv', 39], ['4FGL J2212+2356.csv', 35], ['4FGL J2212+2800.csv', 39], ['4FGL J2216+2421.csv', 37], ['4FGL J2219+1806.csv', 40], ['4FGL J2219-0342.csv', 41], ['4FGL J2225-0457.csv', 45], ['4FGL J2227+0036.csv', 37], ['4FGL J2228-1636.csv', 49], ['4FGL J2229-0832.csv', 37], ['4FGL J2232+1143.csv', 51], ['4FGL J2236+2828.

In [15]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f) and filename in errOFF :
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
        
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)

In [16]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON

In [17]:

from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore= []
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):

#         a=Q_ON2[j]

    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)

for i in range(iteration):
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))


gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]

if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/5):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

(124, 3, 37)
Accuracy for ON class:  100.0 %
Accuracy for OFF class:  85.79759892010817 %
False Positive rate:  14.202401079891825 %
False Negative rate:  0.0 %
F1 score:  0.900423406593497
[['4FGL J2329+6101.csv', 47], ['4FGLJ0018+2946.csv', 53], ['4FGLJ0019+7327.csv', 44], ['4FGLJ0028+2001.csv', 42], ['4FGLJ0033-1921.csv', 41]]


In [18]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f) and filename in errOFF :
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
        
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)

In [19]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON

In [20]:

from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):


    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    if off_nbs>0: 
        OFF_accuracy.append(100*(off_score/off_nbs))
        FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]

if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>1:
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

(35, 3, 37)
Accuracy for ON class:  64.44761904761904 %
Accuracy for OFF class:  82.5925925925926 %
False Positive rate:  17.40740740740741 %
False Negative rate:  35.55238095238095 %
F1 score:  0.7270905483405484
[]
